### PyTorch


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# Transforms to tensor and normalize the data
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
    ])

In [4]:
# Load the MNIST dataset
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 225492936.99it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 114494984.71it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 64081125.58it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 22869782.43it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# Dataloaders for train and test
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [7]:
# Initialize the model
model = CNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%")

    # Check if the desired accuracy is achieved
    if accuracy >= 99.40:
        break

Epoch [1/10], Validation Accuracy: 98.40%
Epoch [2/10], Validation Accuracy: 98.80%
Epoch [3/10], Validation Accuracy: 98.99%
Epoch [4/10], Validation Accuracy: 98.88%
Epoch [5/10], Validation Accuracy: 99.02%
Epoch [6/10], Validation Accuracy: 99.05%
Epoch [7/10], Validation Accuracy: 98.85%
Epoch [8/10], Validation Accuracy: 99.03%
Epoch [9/10], Validation Accuracy: 98.93%
Epoch [10/10], Validation Accuracy: 99.01%


### Tensorflow

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0


# Define the CNN model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=1)

    # Evaluation
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy*100:.2f}%")

    # Check if the desired accuracy is achieved
    if accuracy >= 0.994:
        break


938/938 [==============================] - 7s 4ms/step - loss: 0.1851 - accuracy: 0.9447
Epoch [1/10], Validation Accuracy: 98.25%
938/938 [==============================] - 3s 4ms/step - loss: 0.0576 - accuracy: 0.9826
Epoch [2/10], Validation Accuracy: 98.45%
938/938 [==============================] - 4s 4ms/step - loss: 0.0424 - accuracy: 0.9866
Epoch [3/10], Validation Accuracy: 98.70%
938/938 [==============================] - 3s 4ms/step - loss: 0.0323 - accuracy: 0.9898
Epoch [4/10], Validation Accuracy: 98.86%
938/938 [==============================] - 4s 4ms/step - loss: 0.0246 - accuracy: 0.9924
Epoch [5/10], Validation Accuracy: 98.98%
938/938 [==============================] - 3s 4ms/step - loss: 0.0205 - accuracy: 0.9934
Epoch [6/10], Validation Accuracy: 99.01%
938/938 [==============================] - 4s 4ms/step - loss: 0.0165 - accuracy: 0.9946
Epoch [7/10], Validation Accuracy: 99.17%
938/938 [==============================] - 3s 4ms/step - loss: 0.0136 - accuracy: 0